In [2]:
import torch

# モデルのクラスが定義されていないとloadできないため
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.l1 = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        x = self.l1(x)
        return x

# 52で学習したモデルのロード
model = torch.load('model_weight.pth')

In [21]:
from tqdm import tqdm

# bag of wordsの辞書を作成
def create_BoW_dict(file_path):
    BoW_dict = {}
    i = 0
    with open(file_path, "r", encoding="utf-8")as f:
        data_list = f.readlines()
        for data in data_list:
            data = data.split('\t')
            word_list = data[0].split()
            for word in word_list:
                if word in BoW_dict:
                    continue
                else:
                    BoW_dict[word] = i
                    i += 1
    return BoW_dict

TRAIN_FEATURE_FILE_PATH = "./news+aggregator/train.feature.txt"

# bag of wordsの辞書型を作成
BoW_dict = create_BoW_dict(TRAIN_FEATURE_FILE_PATH)

In [28]:
import numpy as np
weight = model.state_dict()["l1.weight"]
abs_tensor = torch.abs(weight)  #重みを全て絶対値を取る
flat_abs_tensor = abs_tensor.view(-1)     #tensorの形状を変更する．-1とするとflatになる．
top10_weight, top10_index = torch.topk(flat_abs_tensor, k=10)
bottom10_weight, bottom10_index = torch.topk(flat_abs_tensor, k=10, largest=False)
top10_index = top10_index%19433 #特徴ベクトルの次元数でわる
bottom10_index = bottom10_index%19433
top10_feature_list = []
for index in top10_index:
    key = [k for k,v in BoW_dict.items() if v == index]
    top10_feature_list.append(key[0])

    bottom10_feature_list = []

for index in bottom10_index:
    key = [k for k,v in BoW_dict.items() if v == index]
    bottom10_feature_list.append(key[0])

# 単語の影響の大小を表している． 例えばEbolaは最も影響のある特徴量である．
print(f"top10の特徴量: {top10_feature_list}")
print(f"bottom10の特徴量: {bottom10_feature_list}")

top10の特徴量: ['Ebola', 'Facebook', 'Google', 'FDA', 'MERS', 'Bank', 'Microsoft', 'Kardashian', 'Apple', 'FCC']
bottom10の特徴量: ['Announced', 'Understand', 'tuxedo', 'Vladimir', 'AheadDoubledigit', 'Flag', 'Athena', 'Auditions', 'Masters', 'Bonaroo']
